In [1]:
# Imports
import torch
cuda = torch.cuda.is_available()
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
plt.switch_backend("agg")
import sys
sys.path.append("../../semi-supervised")

torch.multiprocessing.set_sharing_strategy('file_system')

from models import AuxiliaryDeepGenerativeModel, DeepGenerativeModel, StackedDeepGenerativeModel, VariationalAutoencoder

In [2]:
features = VariationalAutoencoder([64 * 64 * 3, 300, [300]], conv=True, batch_norm=False).cuda()
features.load_state_dict(torch.load("./vae_celeba_new.ckpt"))
stacked = StackedDeepGenerativeModel([64 * 64 * 3, 4, 50, [500]], features, batch_norm=False)
stacked.dgm.load_state_dict(torch.load("./m1m2_celeba_new.ckpt"))
stacked.dgm = stacked.dgm.cuda()

adgm = AuxiliaryDeepGenerativeModel([64 * 64 * 3, 4, 300, 300, [1000, 1000]], conv=True, batch_norm=False)
adgm.load_state_dict(torch.load("./adgm_celeba_new.ckpt"))
adgm = adgm.cuda()




../../semi-supervised/models/vae.py:237: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data)


300
1024


../../semi-supervised/models/dgm.py:109: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight.data)


[1024, 300]


In [3]:
from datautils import get_celeba

labelled, unlabelled, validation, labels_names = get_celeba(location="./", batch_size=500, labels_per_class=100)

In [8]:
from tqdm import tqdm_notebook
adgm.eval()

correct_preds = 0
for x, y in tqdm_notebook(validation):

    if cuda:
        x, y = x.cuda(device=0), y.cuda(device=0)
        
    # print(y)

    # x, _, _ = features.encoder(x)
    x = x.repeat(1, 1, 1 ,1)
    logits = adgm.classify(x)
    logits = logits.reshape(1, -1, logits.shape[-1]).mean(0)
    _, pred_idx = torch.max(logits, 1)
    _, lab_idx = torch.max(y, 1)
    correct_preds += torch.sum((torch.max(logits, 1)[1].data == torch.max(y, 1)[1].data).float())

accuracy = (correct_preds / len(validation.dataset)).item()
print("ADGM Accuracy {:.3f}\tError: {:.3f}".format(accuracy, 1 - accuracy))

stacked.features.eval()
stacked.dgm.eval()
correct_preds = 0
for x, y in tqdm_notebook(validation):

    if cuda:
        x, y = x.cuda(device=0), y.cuda(device=0)
        
    # print(y)

    x, _, _ = stacked.features.encode(x)
    logits = stacked.dgm.classify(x)
    _, pred_idx = torch.max(logits, 1)
    _, lab_idx = torch.max(y, 1)
    correct_preds += torch.sum((torch.max(logits, 1)[1].data == torch.max(y, 1)[1].data).float())

accuracy = (correct_preds / len(validation.dataset)).item()
print("Stacked Accuracy {:.3f}\tError: {:.3f}".format(accuracy, 1 - accuracy))


ADGM Accuracy 0.583	Error: 0.417


Stacked Accuracy 0.555	Error: 0.445


In [9]:
from metrics import sample_from_classes, interpolation, cyclic_interpolation, save_samples
from tqdm import trange

adgm.eval()
stacked.dgm.eval()
stacked.features.eval()

im_shape = [64, 64, 3]
classes_num = 4
z_dim = 300
labels_names = ["F/NS", "F/S", "M/NS", "M/S"]



In [10]:
cyclic_interpolation("adgm_celeba", adgm, validation.dataset, im_shape, classes_num, labels_names)
cyclic_interpolation("m1m2_celeba", stacked, validation.dataset, im_shape, classes_num, labels_names)

interpolation("adgm_celeba", adgm, validation.dataset, im_shape)
interpolation("m1m2_celeba", stacked, validation.dataset, im_shape)

sample_from_classes("adgm_celeba", adgm, im_shape, 300, classes_num)
sample_from_classes("m1m2_celeba", stacked, im_shape, 50, classes_num)


/home/mwolczyk/miniconda3/envs/pytorch_nightly/lib/python3.7/site-packages/torch/nn/functional.py:1382: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/mwolczyk/miniconda3/envs/pytorch_nightly/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [11]:
save_samples("adgm_celeba", adgm, im_shape, 10000, classes_num, 300)
save_samples("m1m2_celeba", stacked, im_shape, 10000, classes_num, 50)

100%|██████████| 4/4 [00:14<00:00,  3.75s/it]


# 